In [1]:
import torch
import numpy as np
import random
from transformer_lens import HookedTransformer, ActivationCache
import transformer_lens.utils as utils
import plotly.express as px


SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# %%
model = HookedTransformer.from_pretrained('gpt2', dtype=torch.float16)

Loaded pretrained model gpt2 into HookedTransformer


In [17]:
from atp import Patching
model.eval()
model.set_use_attn_result(True)
model.set_use_attn_in(True)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)

from importlib import reload
import atp
reload(atp)

patching = atp.Patching(model, 'ap')

In [3]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu", "high_school_biology")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Generating test split:   0%|          | 0/310 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/32 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [7]:
letters = ['A', 'B', 'C', 'D']
prelude = "A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. "
post_text = "Answer: ("

def get_prompt_pair(i):
    correct_id = dataset["test"][i]["answer"]
    question = dataset["test"][i]["question"] + ':\n'

    clean_answers = f"""
(A) {dataset["test"][i]["choices"][0][:-1]}
(B) {dataset["test"][i]["choices"][1][:-1]}
(C) {dataset["test"][i]["choices"][2][:-1]}
(D) {dataset["test"][i]["choices"][3][:-1]}
"""

    corrupted_answers = f"""
(A) {dataset["test"][i]["choices"][1][:-1]}
(B) {dataset["test"][i]["choices"][2][:-1]}
(C) {dataset["test"][i]["choices"][3][:-1]}
(D) {dataset["test"][i]["choices"][0][:-1]}
"""

    clean = prelude + question + clean_answers + post_text
    corrupted = prelude + question + corrupted_answers + post_text

    return clean, corrupted, letters[correct_id]

In [8]:
x_clean, x_corr, a_clean = get_prompt_pair(0)
print(x_clean)
print(x_corr)
print(a_clean)

A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. In a population of giraffes, an environmental change occurs that favors individuals that are tallest. As a result, more of the taller individuals are able to obtain nutrients and survive to pass along their genetic information. This is an example of:

(A) directional selection
(B) stabilizing selection
(C) sexual selection
(D) disruptive selection
Answer: (
A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. In a population of giraffes, an environmental change occurs that favors individuals that are tallest. As a result, more of the taller individuals are able to obtain nutrients and survive to pass along their genetic information. This is an example of:

(A) stabilizing selection
(B) sexual selection
(C) disruptive selection
(D) directional selection
Answer: (
A


In [ ]:
# Check if the model can generate the correct answer
print(model.generate(x_clean), max_new_tokens=1)

In [18]:
patching.patching(x_clean, a_clean, x_corr, "D", component='attn_out') # mlp_post / attn_out

Clean logit difference: 3.055
Corrupted logit difference: 3.094
Patching...


100%|██████████| 12/12 [02:09<00:00, 10.77s/it]


In [19]:
patching.plot_patch(width=1200, height=800)

In [25]:
from logitlens import plot_logit_lens

fig = plot_logit_lens(model, x_clean, what='probs', component='resid_post', tok_id=None, cutoff=35)
fig.update_layout(height=800)
fig.show()

Next token: A tensor(32)
